# **<font color='blue'><u>Big Data Project</u> <br></font>**


<br>
<br>
<hr class="dashed">
<br>
<br>

In [1]:
# Create the enviroment
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null  
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz 
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

import findspark
findspark.init("spark-3.3.0-bin-hadoop3")
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql import Row
from pyspark.sql import functions


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.c

In [2]:
# Create session
spark = SparkSession.builder.appName("Basic").getOrCreate()


In [3]:
#Connect to Gogle Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Import libraries
import pyspark
import pandas as pd
from pyspark.sql.functions import split
import pyspark.sql.functions as f


<br>
<br>
<hr class="dashed">
<br>
<br>

In [6]:
# Import Data Files
Ratings = spark.read.csv('/content/drive/MyDrive/Spark/BigDataProject/BX-Book-Ratings.csv',header=True,inferSchema=True, encoding = "iso-8859-1")
Books = spark.read.csv('/content/drive/MyDrive/Spark/BigDataProject/BX-Books.csv',header=True,inferSchema=True, encoding = "iso-8859-1")
Users = spark.read.csv ('/content/drive/MyDrive/Spark/BigDataProject/BX-Users.csv',header=True,inferSchema=True, encoding = "iso-8859-1")

In [ ]:
Ratings.describe().show()

In [ ]:
Books.describe().show()

In [ ]:
Users.describe().show()

<br>
<br>
<hr class="dashed">
<br>
<br>

# **<font color='ired'><u>Ratings Transformations</u> <br></font>**

In [ ]:
Ratings.printSchema()

root
 |-- "User-ID;""ISBN"";""Book-Rating""": string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [7]:
#spliting the columns
from pyspark.sql.functions import split

split_column = pyspark.sql.functions.split(Ratings['"User-ID;""ISBN"";""Book-Rating"""'], ';')
Ratings2 = Ratings.withColumn('userid', split_column.getItem(0))
Ratings3 = Ratings2.withColumn('isbn', split_column.getItem(1))
Ratings4 = Ratings3.withColumn('rating', split_column.getItem(2))
Ratings5 = Ratings4.drop('_c1', '_c2', '"User-ID;""ISBN"";""Book-Rating"""' )
Ratings5.show()


+-------+--------------+-------+
| userid|          isbn| rating|
+-------+--------------+-------+
|"276725|""034545104X""| ""0"""|
|"276726|""0155061224""| ""5"""|
|"276727|""0446520802""| ""0"""|
|"276729|""052165615X""| ""3"""|
|"276729|""0521795028""| ""6"""|
|"276733|""2080674722""| ""0"""|
|"276736|""3257224281""| ""8"""|
|"276737|""0600570967""| ""6"""|
|"276744|""038550120X""| ""7"""|
|"276745| ""342310538""|""10"""|
|"276746|""0425115801""| ""0"""|
|"276746|""0449006522""| ""0"""|
|"276746|""0553561618""| ""0"""|
|"276746|""055356451X""| ""0"""|
|"276746|""0786013990""| ""0"""|
|"276746|""0786014512""| ""0"""|
|"276747|""0060517794""| ""9"""|
|"276747|""0451192001""| ""0"""|
|"276747|""0609801279""| ""0"""|
|"276747|""0671537458""| ""9"""|
+-------+--------------+-------+
only showing top 20 rows



In [8]:
# cleaning unwanted character '"'

from pyspark.sql.functions import regexp_replace, col
Ratings6 = Ratings5.withColumn('userid', regexp_replace('userid', '"', ''))
Ratings7 = Ratings6.withColumn('isbn', regexp_replace('isbn', '"', ''))
Ratings8 = Ratings7.withColumn('rating', regexp_replace('rating', '"', ''))
Ratings9 = Ratings8.withColumn('rating', regexp_replace('rating', ',', ''))
Ratings10 = Ratings9.na.drop(subset=["rating"])
Ratings11 = Ratings10.withColumn( 'isbn', regexp_replace("isbn", "[\$#,]", ''))

Ratings11.show()

+------+----------+------+
|userid|      isbn|rating|
+------+----------+------+
|276725|034545104X|     0|
|276726|0155061224|     5|
|276727|0446520802|     0|
|276729|052165615X|     3|
|276729|0521795028|     6|
|276733|2080674722|     0|
|276736|3257224281|     8|
|276737|0600570967|     6|
|276744|038550120X|     7|
|276745| 342310538|    10|
|276746|0425115801|     0|
|276746|0449006522|     0|
|276746|0553561618|     0|
|276746|055356451X|     0|
|276746|0786013990|     0|
|276746|0786014512|     0|
|276747|0060517794|     9|
|276747|0451192001|     0|
|276747|0609801279|     0|
|276747|0671537458|     9|
+------+----------+------+
only showing top 20 rows



In [9]:
# Null Check up
Ratings11.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in Ratings11.columns]).show()

+------+----+------+
|userid|isbn|rating|
+------+----+------+
|     0|   0|     0|
+------+----+------+



In [10]:
# changing userid and rating formats from string to int/float
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType

Ratings12 = Ratings11.withColumn("rating", Ratings11["rating"].cast('float'))\
            .withColumn("userid", Ratings11["userid"].cast(IntegerType()))
Ratings12.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- rating: float (nullable = true)



In [ ]:
# Ratings distribution
rating_count = Ratings12.groupBy(["rating"]).count().orderBy(f.desc("rating"))
rating_count.show()

+------+------+
|rating| count|
+------+------+
|  10.0| 71813|
|   9.0| 62101|
|   8.0| 95037|
|   7.0| 70018|
|   6.0| 33752|
|   5.0| 46953|
|   4.0|  8031|
|   3.0|  5445|
|   2.0|  2492|
|   1.0|  1601|
|   0.0|651327|
+------+------+



In [ ]:
# Most active users
grouppedbyuserid = Ratings12.groupBy(Ratings12['userid']).count().orderBy(f.desc("count"))
grouppedbyuserid.show()


+------+-----+
|userid|count|
+------+-----+
| 11676|13602|
|198711| 7550|
|153662| 6109|
| 98391| 5891|
| 35859| 5850|
|212898| 4785|
|278418| 4533|
| 76352| 3367|
|110973| 3100|
|235105| 3067|
|230522| 2991|
| 16795| 2948|
|234623| 2674|
| 36836| 2529|
| 52584| 2512|
|245963| 2507|
|204864| 2504|
| 55492| 2459|
|185233| 2448|
|171118| 2421|
+------+-----+
only showing top 20 rows



In [ ]:
Ratings12.show()

+------+----------+------+
|userid|      isbn|rating|
+------+----------+------+
|276725|034545104X|   0.0|
|276726|0155061224|   5.0|
|276727|0446520802|   0.0|
|276729|052165615X|   3.0|
|276729|0521795028|   6.0|
|276733|2080674722|   0.0|
|276736|3257224281|   8.0|
|276737|0600570967|   6.0|
|276744|038550120X|   7.0|
|276745| 342310538|  10.0|
|276746|0425115801|   0.0|
|276746|0449006522|   0.0|
|276746|0553561618|   0.0|
|276746|055356451X|   0.0|
|276746|0786013990|   0.0|
|276746|0786014512|   0.0|
|276747|0060517794|   9.0|
|276747|0451192001|   0.0|
|276747|0609801279|   0.0|
|276747|0671537458|   9.0|
+------+----------+------+
only showing top 20 rows



In [ ]:
Ratings12.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- rating: float (nullable = true)



In [11]:
# Creatimg dim_ratings
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window

dim_ratings = Ratings12.withColumn(
    "ratingSK",
    row_number().over(Window.orderBy(monotonically_increasing_id()))
)

dim_ratings.show()

+------+----------+------+--------+
|userid|      isbn|rating|ratingSK|
+------+----------+------+--------+
|276725|034545104X|   0.0|       1|
|276726|0155061224|   5.0|       2|
|276727|0446520802|   0.0|       3|
|276729|052165615X|   3.0|       4|
|276729|0521795028|   6.0|       5|
|276733|2080674722|   0.0|       6|
|276736|3257224281|   8.0|       7|
|276737|0600570967|   6.0|       8|
|276744|038550120X|   7.0|       9|
|276745| 342310538|  10.0|      10|
|276746|0425115801|   0.0|      11|
|276746|0449006522|   0.0|      12|
|276746|0553561618|   0.0|      13|
|276746|055356451X|   0.0|      14|
|276746|0786013990|   0.0|      15|
|276746|0786014512|   0.0|      16|
|276747|0060517794|   9.0|      17|
|276747|0451192001|   0.0|      18|
|276747|0609801279|   0.0|      19|
|276747|0671537458|   9.0|      20|
+------+----------+------+--------+
only showing top 20 rows



In [12]:
# rearrange columns
dim_ratings = dim_ratings.select("ratingSK","userid","isbn", "rating")
dim_ratings.show()

+--------+------+----------+------+
|ratingSK|userid|      isbn|rating|
+--------+------+----------+------+
|       1|276725|034545104X|   0.0|
|       2|276726|0155061224|   5.0|
|       3|276727|0446520802|   0.0|
|       4|276729|052165615X|   3.0|
|       5|276729|0521795028|   6.0|
|       6|276733|2080674722|   0.0|
|       7|276736|3257224281|   8.0|
|       8|276737|0600570967|   6.0|
|       9|276744|038550120X|   7.0|
|      10|276745| 342310538|  10.0|
|      11|276746|0425115801|   0.0|
|      12|276746|0449006522|   0.0|
|      13|276746|0553561618|   0.0|
|      14|276746|055356451X|   0.0|
|      15|276746|0786013990|   0.0|
|      16|276746|0786014512|   0.0|
|      17|276747|0060517794|   9.0|
|      18|276747|0451192001|   0.0|
|      19|276747|0609801279|   0.0|
|      20|276747|0671537458|   9.0|
+--------+------+----------+------+
only showing top 20 rows



<br>
<br>
<hr class="dashed">
<br>
<br>

# **<font color='ired'><u>Books Transformations</u> <br></font>**

In [ ]:
Books.printSchema()

root
 |-- "ISBN";"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L": string (nullable = true)



In [ ]:
Books.show()

+-------------------------------------------------------------------------------------------------------------+
|"ISBN";"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"|
+-------------------------------------------------------------------------------------------------------------+
|                                                                                         "0195153448";"Cla...|
|                                                                                         "0002005018";"Cla...|
|                                                                                         "0060973129";"Dec...|
|                                                                                         "0374157065";"Flu...|
|                                                                                         "0393045218";"The...|
|                                                                                         "0399135782";"

In [13]:
#splitting the columns
from pyspark.sql.functions import split

split_column1 = pyspark.sql.functions.split(Books['"ISBN";"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"'], ';')
Books1 = Books.withColumn('isbn', split_column1.getItem(0))\
              .withColumn('title', split_column1.getItem(1))\
              .withColumn('author', split_column1.getItem(2))\
              .withColumn('year', split_column1.getItem(3))\
              .withColumn('publisher', split_column1.getItem(4))

Books2 = Books1.drop('"ISBN";"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"')
Books2.show()

+------------+--------------------+--------------------+------+--------------------+
|        isbn|               title|              author|  year|           publisher|
+------------+--------------------+--------------------+------+--------------------+
|"0195153448"|"Classical Mythol...|"Mark P. O. Morford"|"2002"|"Oxford Universit...|
|"0002005018"|      "Clara Callan"|"Richard Bruce Wr...|"2001"|"HarperFlamingo C...|
|"0060973129"|"Decision in Norm...|      "Carlo D'Este"|"1991"|   "HarperPerennial"|
|"0374157065"|"Flu: The Story o...|  "Gina Bari Kolata"|"1999"|"Farrar Straus Gi...|
|"0393045218"|"The Mummies of U...|   "E. J. W. Barber"|"1999"|  "W. W. Norton &amp|
|"0399135782"|"The Kitchen God'...|           "Amy Tan"|"1991"|  "Putnam Pub Group"|
|"0425176428"|"What If?: The Wo...|     "Robert Cowley"|"2000"|"Berkley Publishi...|
|"0671870432"|   "PLEADING GUILTY"|       "Scott Turow"|"1993"|        "Audioworks"|
|"0679425608"|"Under the Black ...|   "David Cordingly"|"1996"|  

In [14]:
# cleaning unwanted characters

from pyspark.sql.functions import regexp_replace, col

Books3 = Books2.withColumn('isbn', regexp_replace('isbn', '"', ''))\
                .withColumn('title', regexp_replace('title', '"', ''))\
                .withColumn('author', regexp_replace('author', '"', ''))\
                .withColumn('year', regexp_replace('year', '"', ''))\
                .withColumn('publisher', regexp_replace('publisher', '"', ''))

Books4 = Books3.na.fill(value='unknown',subset=["author"])


Books4.show()


+----------+--------------------+--------------------+----+--------------------+
|      isbn|               title|              author|year|           publisher|
+----------+--------------------+--------------------+----+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|2002|Oxford University...|
|0002005018|        Clara Callan|Richard Bruce Wright|2001|HarperFlamingo Ca...|
|0060973129|Decision in Normandy|        Carlo D'Este|1991|     HarperPerennial|
|0374157065|Flu: The Story of...|    Gina Bari Kolata|1999|Farrar Straus Giroux|
|0393045218|The Mummies of Ur...|     E. J. W. Barber|1999|   W. W. Norton &amp|
|0399135782|The Kitchen God's...|             Amy Tan|1991|    Putnam Pub Group|
|0425176428|What If?: The Wor...|       Robert Cowley|2000|Berkley Publishin...|
|0671870432|     PLEADING GUILTY|         Scott Turow|1993|          Audioworks|
|0679425608|Under the Black F...|     David Cordingly|1996|        Random House|
|074322678X|Where You'll Fin

In [ ]:
# Null check up: there are nulls in year and publisher columns
Books4.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in Books4.columns]).show()

+----+-----+------+-----+---------+
|isbn|title|author| year|publisher|
+----+-----+------+-----+---------+
|   0|    0|     0|38846|    38965|
+----+-----+------+-----+---------+



In [15]:
# Replace null in publisher with unknown
Books5 = Books4.fillna('unknown', ['publisher'])


In [ ]:
# data in publisher column contains years. year column contains authors, some book names are splited between title and author columns. we'll try to fix it. 
Books5.filter('publisher like "19%"').show()


+----------+--------------------+--------------------+----------------+---------+
|      isbn|               title|              author|            year|publisher|
+----------+--------------------+--------------------+----------------+---------+
|0679751343|         Angels &amp| Insects : Two No...|      A.S. BYATT|     1994|
|0843136685|        Sheldon &amp| Mrs. Levine: An ...|     Sam Bobrick|     1994|
|1567920047|A Tally of Types:...| And With a New I...| Stanley Morison|     1999|
|0793133955|Wall Street's Pic...| Mutual Funds (Wa...|  Kirk Kazanjian|     1999|
|0446604801|           Jack &amp| Jill (Alex Cross...| James Patterson|     1997|
|0553574833|So That Others Ma...| Her Search-And-R...| Hank Whittemore|     1996|
|015644450X|        In Love &amp| Trouble: Stories...|    Alice Walker|     1974|
|2070372294|Le Coq De Bruygre...|          No. 1229))|        Tournier|     1987|
|1558506454|Museum of Science...|           Questions|   Ann Rae Jonas|     1996|
|0446673544|    

In [16]:
# Year solution

Books6 = Books5.withColumn('year1', f.when(
                          Books5.publisher.like("19%") | Books5.publisher.like("20%") ,Books5.publisher)\
                           .otherwise(Books5.year))
Books6.show()

+----------+--------------------+--------------------+----+--------------------+-----+
|      isbn|               title|              author|year|           publisher|year1|
+----------+--------------------+--------------------+----+--------------------+-----+
|0195153448| Classical Mythology|  Mark P. O. Morford|2002|Oxford University...| 2002|
|0002005018|        Clara Callan|Richard Bruce Wright|2001|HarperFlamingo Ca...| 2001|
|0060973129|Decision in Normandy|        Carlo D'Este|1991|     HarperPerennial| 1991|
|0374157065|Flu: The Story of...|    Gina Bari Kolata|1999|Farrar Straus Giroux| 1999|
|0393045218|The Mummies of Ur...|     E. J. W. Barber|1999|   W. W. Norton &amp| 1999|
|0399135782|The Kitchen God's...|             Amy Tan|1991|    Putnam Pub Group| 1991|
|0425176428|What If?: The Wor...|       Robert Cowley|2000|Berkley Publishin...| 2000|
|0671870432|     PLEADING GUILTY|         Scott Turow|1993|          Audioworks| 1993|
|0679425608|Under the Black F...|     David

In [17]:
# Publisher solution
Books7 = Books6.withColumn('publisher1', f.when(
                          Books6.publisher.like("19%") | Books6.publisher.like("20%") ,'unknown')\
                           .otherwise(Books6.publisher))
Books7.show()

+----------+--------------------+--------------------+----+--------------------+-----+--------------------+
|      isbn|               title|              author|year|           publisher|year1|          publisher1|
+----------+--------------------+--------------------+----+--------------------+-----+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|2002|Oxford University...| 2002|Oxford University...|
|0002005018|        Clara Callan|Richard Bruce Wright|2001|HarperFlamingo Ca...| 2001|HarperFlamingo Ca...|
|0060973129|Decision in Normandy|        Carlo D'Este|1991|     HarperPerennial| 1991|     HarperPerennial|
|0374157065|Flu: The Story of...|    Gina Bari Kolata|1999|Farrar Straus Giroux| 1999|Farrar Straus Giroux|
|0393045218|The Mummies of Ur...|     E. J. W. Barber|1999|   W. W. Norton &amp| 1999|   W. W. Norton &amp|
|0399135782|The Kitchen God's...|             Amy Tan|1991|    Putnam Pub Group| 1991|    Putnam Pub Group|
|0425176428|What If?: The Wo

In [ ]:
Books7.describe().show()

+-------+--------------------+--------------------+------------------+-------------------+------------------+------------------+------------------+
|summary|                isbn|               title|            author|               year|         publisher|             year1|        publisher1|
+-------+--------------------+--------------------+------------------+-------------------+------------------+------------------+------------------+
|  count|              271379|              271379|            271379|             232533|            271379|            232533|            271379|
|   mean|1.0412353852604752E9|            Infinity|               1.5| 1954.7740462835868|1997.8285575522098|1955.4417198001033|2781.3703703703704|
| stddev|1.4877523499740484E9|                 NaN|0.7071067811865476|   275.652366164257| 798.9126693408563|273.26070600367586|10020.520975816607|
|    min|          0000913154| A Light in the S...|                  |                   |                  |   

In [18]:
# Empties in publisher1 column
Books7.filter(Books7.publisher1 == '').show()

+----------+------------------+-------------+--------------+---------+--------------+----------+
|      isbn|             title|       author|          year|publisher|         year1|publisher1|
+----------+------------------+-------------+--------------+---------+--------------+----------+
|0887231241|The Big Black Book|Boardroom&amp|S Experts &amp|         |S Experts &amp|          |
+----------+------------------+-------------+--------------+---------+--------------+----------+



In [19]:
# Relacing empties with unlnown in publiser1
from pyspark.sql.functions import regexp_replace
Books8 = Books7.withColumn('publisher1', f.when(Books7.publisher1 == '' , regexp_replace(Books7.publisher1,'','unknown')).otherwise(Books7.publisher1))


In [ ]:
Books8.describe().show()

+-------+--------------------+--------------------+------------------+-------------------+------------------+------------------+--------------------+
|summary|                isbn|               title|            author|               year|         publisher|             year1|          publisher1|
+-------+--------------------+--------------------+------------------+-------------------+------------------+------------------+--------------------+
|  count|              271379|              271379|            271379|             232533|            271379|            232533|              271379|
|   mean|1.0412353852604752E9|            Infinity|               1.5| 1954.7740462835868|1997.8285575522098|1955.4417198001033|  2781.3703703703704|
| stddev|1.4877523499740484E9|                 NaN|0.7071067811865476|   275.652366164257| 798.9126693408563|273.26070600367586|  10020.520975816607|
|    min|          0000913154| A Light in the S...|                  |                   |          

In [ ]:
# 10 empty spaces in year1 column
Books8.filter("year1 = ''").show()

+----------+--------------------+--------------------+----+----------------+-----+----------------+
|      isbn|               title|              author|year|       publisher|year1|      publisher1|
+----------+--------------------+--------------------+----+----------------+-----+----------------+
|0385272170|The Semi-Attached...|               [&amp|    |         unknown|     |         unknown|
|0704300052|     Waterside birds|: Beautiful colou...|    |Nicholas Hammond|     |Nicholas Hammond|
|0600361721|      Marilyn Monroe|    : A life on film|    |      John Kobal|     |      John Kobal|
|0002255464|            Goblins |              : &amp|    |         unknown|     |         unknown|
+----------+--------------------+--------------------+----+----------------+-----+----------------+



In [ ]:
# nulls in year1 column
Books8.filter(Books8.year1.isNull()).count()

38846

In [20]:
# Replace null in year1 column with 9999 - because 1900 may be an actual year in data
Books9 = Books8.fillna("9999",["year1"])
Books9.filter(Books9.year1.isNull()).count()

0

In [21]:
# Replace empties in year1 column with 9999
Books10 = Books9.withColumn('year1', f.when(Books9.year1 == '' , regexp_replace(Books9.year1,'','9999')).otherwise(Books9.year1))
Books10.filter("year1 = ''").show()

+----+-----+------+----+---------+-----+----------+
|isbn|title|author|year|publisher|year1|publisher1|
+----+-----+------+----+---------+-----+----------+
+----+-----+------+----+---------+-----+----------+



In [ ]:
# years beyond current year
Books10.filter("year1  > 2022 and year1 !=9999").show()

+----------+--------------------+--------------------+----+-------------------+-----+-------------------+
|      isbn|               title|              author|year|          publisher|year1|         publisher1|
+----------+--------------------+--------------------+----+-------------------+-----+-------------------+
|0671746103|MY TEACHER FRIED ...|             Coville|2030|            Aladdin| 2030|            Aladdin|
|0671791990|MY TEACHER FLUNKE...|       Bruce Coville|2030|            Aladdin| 2030|            Aladdin|
|0870449842|    Crossing America|National Geograph...|2030|National Geographic| 2030|National Geographic|
|0140301690|Alice's Adventure...|       Lewis Carroll|2050|       Puffin Books| 2050|       Puffin Books|
|0140201092|Outline of Europe...|    Nikolaus Pevsner|2050|        Penguin USA| 2050|        Penguin USA|
|0394701658|Three Plays of Eu...|      Eugene O'Neill|2038|  Vintage Books USA| 2038|  Vintage Books USA|
|3442436893|Das groÃ?Â?e BÃ?Â...|         Kath

In [22]:
# Replace future years in year1 column with 9999
Books10 = Books10.withColumn('year1', f.when(col("year1") > 2022,'9999').otherwise(Books10.year1))
Books10.filter("year1  > 2022 and year1 !=9999").show()

+----+-----+------+----+---------+-----+----------+
|isbn|title|author|year|publisher|year1|publisher1|
+----+-----+------+----+---------+-----+----------+
+----+-----+------+----+---------+-----+----------+



In [ ]:
# Nulls check up
Books10.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in Books10.columns]).show()

+----+-----+------+-----+---------+-----+----------+
|isbn|title|author| year|publisher|year1|publisher1|
+----+-----+------+-----+---------+-----+----------+
|   0|    0|     0|38846|        0|    0|         0|
+----+-----+------+-----+---------+-----+----------+



In [ ]:
Books10.describe().show()

+-------+--------------------+--------------------+------------------+-------------------+------------------+--------------------+--------------------+
|summary|                isbn|               title|            author|               year|         publisher|               year1|          publisher1|
+-------+--------------------+--------------------+------------------+-------------------+------------------+--------------------+--------------------+
|  count|              271379|              271379|            271379|             232533|            271379|              271379|              271379|
|   mean|1.0412353852604752E9|            Infinity|               1.5| 1954.7740462835868|1997.8285575522098|   3109.030021773628|  2781.3703703703704|
| stddev|1.4877523499740484E9|                 NaN|0.7071067811865476|   275.652366164257| 798.9126693408563|  2830.5849518204554|  10020.520975816607|
|    min|          0000913154| A Light in the S...|                  |                  

In [23]:
#Changing format year1 to int
Books11 = Books10.withColumn("year1", Books10["year1"].cast(IntegerType()))
Books11.show()

+----------+--------------------+--------------------+----+--------------------+-----+--------------------+
|      isbn|               title|              author|year|           publisher|year1|          publisher1|
+----------+--------------------+--------------------+----+--------------------+-----+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|2002|Oxford University...| 2002|Oxford University...|
|0002005018|        Clara Callan|Richard Bruce Wright|2001|HarperFlamingo Ca...| 2001|HarperFlamingo Ca...|
|0060973129|Decision in Normandy|        Carlo D'Este|1991|     HarperPerennial| 1991|     HarperPerennial|
|0374157065|Flu: The Story of...|    Gina Bari Kolata|1999|Farrar Straus Giroux| 1999|Farrar Straus Giroux|
|0393045218|The Mummies of Ur...|     E. J. W. Barber|1999|   W. W. Norton &amp| 1999|   W. W. Norton &amp|
|0399135782|The Kitchen God's...|             Amy Tan|1991|    Putnam Pub Group| 1991|    Putnam Pub Group|
|0425176428|What If?: The Wo

In [ ]:
Books11.describe().show()

+-------+--------------------+--------------------+------------------+-------------------+------------------+------------------+--------------------+
|summary|                isbn|               title|            author|               year|         publisher|             year1|          publisher1|
+-------+--------------------+--------------------+------------------+-------------------+------------------+------------------+--------------------+
|  count|              271379|              271379|            271379|             232533|            271379|            270970|              271379|
|   mean|1.0412353852604752E9|            Infinity|               1.5| 1954.7740462835868|1997.8285575522098| 3109.030021773628|  2781.3703703703704|
| stddev|1.4877523499740484E9|                 NaN|0.7071067811865476|   275.652366164257| 798.9126693408563|2830.5849518204554|  10020.520975816607|
|    min|          0000913154| A Light in the S...|                  |                   |          

In [24]:
# There are zeros in year1 column
Books11.filter("year1 == 0").count()

4441

In [ ]:
# Looking for "strange" year1 values
Books11.filter("year1 > 0 and year1 < 1900 ").show()

+----------+--------------------+--------------------+----+-------------------+-----+-------------------+
|      isbn|               title|              author|year|          publisher|year1|         publisher1|
+----------+--------------------+--------------------+----+-------------------+-----+-------------------+
|9643112136|Dalan-i bihisht (...|         Nazi Safavi|1378|Intisharat-i Quqnus| 1378|Intisharat-i Quqnus|
|964442011X|        Tasht-i khun|      IsmaÂ°il Fasih|1376|     Nashr-i Alburz| 1376|     Nashr-i Alburz|
|0706407849|Around the World ...| From the Earth t...|  20|            unknown|   20|            unknown|
+----------+--------------------+--------------------+----+-------------------+-----+-------------------+



In [25]:
# Replacing year1 values that earlier than 1000 to 9999
Books12 = Books11.withColumn("year1", f.when(col("year1") < 1000,'9999').otherwise(col("year1")))
Books12.filter("year1 > 0 and year1 < 1900 ").show()

+----------+--------------------+--------------+----+-------------------+-----+-------------------+
|      isbn|               title|        author|year|          publisher|year1|         publisher1|
+----------+--------------------+--------------+----+-------------------+-----+-------------------+
|9643112136|Dalan-i bihisht (...|   Nazi Safavi|1378|Intisharat-i Quqnus| 1378|Intisharat-i Quqnus|
|964442011X|        Tasht-i khun|IsmaÂ°il Fasih|1376|     Nashr-i Alburz| 1376|     Nashr-i Alburz|
+----------+--------------------+--------------+----+-------------------+-----+-------------------+



In [26]:
# Fixing the problem of book names split betweem title and author columns
from pyspark.sql.functions import concat_ws,col

Books13 = Books12.withColumn('title1', f.when(
                          ~ Books12.year.like("19%") & ~ Books12.year.like("20%") ,concat_ws( ' ', Books12.title , Books12.author))\
                           .otherwise(Books12.title))
Books13.filter('year like "20%"').show()

+----------+--------------------+--------------------+----+--------------------+-----+--------------------+--------------------+
|      isbn|               title|              author|year|           publisher|year1|          publisher1|              title1|
+----------+--------------------+--------------------+----+--------------------+-----+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|2002|Oxford University...| 2002|Oxford University...| Classical Mythology|
|0002005018|        Clara Callan|Richard Bruce Wright|2001|HarperFlamingo Ca...| 2001|HarperFlamingo Ca...|        Clara Callan|
|0425176428|What If?: The Wor...|       Robert Cowley|2000|Berkley Publishin...| 2000|Berkley Publishin...|What If?: The Wor...|
|074322678X|Where You'll Find...|         Ann Beattie|2002|            Scribner| 2002|            Scribner|Where You'll Find...|
|080652121X|Hitler's Secret B...|          Adam Lebor|2000|       Citadel Press| 2000|       Cita

In [27]:
# fixing author column
Books14 = Books13.withColumn('author1', f.when(
                         ~ Books13.year.like("19%") & ~ Books13.year.like("20%")  ,Books13.year)\
                           .otherwise(Books13.author))

Books14.filter('publisher like "19%"').show()

+----------+--------------------+--------------------+----------------+---------+-----+----------+--------------------+----------------+
|      isbn|               title|              author|            year|publisher|year1|publisher1|              title1|         author1|
+----------+--------------------+--------------------+----------------+---------+-----+----------+--------------------+----------------+
|0679751343|         Angels &amp| Insects : Two No...|      A.S. BYATT|     1994| 1994|   unknown|Angels &amp  Inse...|      A.S. BYATT|
|0843136685|        Sheldon &amp| Mrs. Levine: An ...|     Sam Bobrick|     1994| 1994|   unknown|Sheldon &amp  Mrs...|     Sam Bobrick|
|1567920047|A Tally of Types:...| And With a New I...| Stanley Morison|     1999| 1999|   unknown|A Tally of Types:...| Stanley Morison|
|0793133955|Wall Street's Pic...| Mutual Funds (Wa...|  Kirk Kazanjian|     1999| 1999|   unknown|Wall Street's Pic...|  Kirk Kazanjian|
|0446604801|           Jack &amp| Jill (A

In [ ]:
# empties in author1 column
Books14.filter('author1 == ""').show()

+----------+--------------------+--------------------+----+----------------+-----+----------------+--------------------+-------+
|      isbn|               title|              author|year|       publisher|year1|      publisher1|              title1|author1|
+----------+--------------------+--------------------+----+----------------+-----+----------------+--------------------+-------+
|0385272170|The Semi-Attached...|               [&amp|    |         unknown| 9999|         unknown|The Semi-Attached...|       |
|0704300052|     Waterside birds|: Beautiful colou...|    |Nicholas Hammond| 9999|Nicholas Hammond|Waterside birds :...|       |
|1590510917|Monsieur Ibrahim ...|                    |null|         unknown| 9999|         unknown|Monsieur Ibrahim ...|       |
|0600361721|      Marilyn Monroe|    : A life on film|    |      John Kobal| 9999|      John Kobal|Marilyn Monroe : ...|       |
|0451626370|Mentor Book of Ma...|        Williams&amp|    |            1993| 1993|         unknow

In [28]:
# replacing empties in authir1 column with 'unknown'
Books15 = Books14.withColumn("author1", f.when(col("author1")=="" ,'unknown').otherwise(col("author1")))

Books15.filter('author1 == ""').show()

+----+-----+------+----+---------+-----+----------+------+-------+
|isbn|title|author|year|publisher|year1|publisher1|title1|author1|
+----+-----+------+----+---------+-----+----------+------+-------+
+----+-----+------+----+---------+-----+----------+------+-------+



In [29]:
# replacing "&amp" with 'and'
Books16 = Books15.withColumn('title1', regexp_replace('title1', '&amp', 'and'))
Books16.filter('title1 like "%&amp%"').show()

+----+-----+------+----+---------+-----+----------+------+-------+
|isbn|title|author|year|publisher|year1|publisher1|title1|author1|
+----+-----+------+----+---------+-----+----------+------+-------+
+----+-----+------+----+---------+-----+----------+------+-------+



In [30]:
# "&amp" value in ublisher1 column
Books16.filter('publisher1 like "%&amp%"').show()

+----------+--------------------+------------------+----+------------------+-----+------------------+--------------------+------------------+
|      isbn|               title|            author|year|         publisher|year1|        publisher1|              title1|           author1|
+----------+--------------------+------------------+----+------------------+-----+------------------+--------------------+------------------+
|0393045218|The Mummies of Ur...|   E. J. W. Barber|1999| W. W. Norton &amp| 1999| W. W. Norton &amp|The Mummies of Ur...|   E. J. W. Barber|
|1841721522|New Vegetarian: B...|Celia Brooks Brown|2001|Ryland Peters &amp| 2001|Ryland Peters &amp|New Vegetarian: B...|Celia Brooks Brown|
|0446310786|To Kill a Mocking...|        Harper Lee|1988| Little Brown &amp| 1988| Little Brown &amp|To Kill a Mocking...|        Harper Lee|
|0340767936|      Turning Thirty|        Mike Gayle|2000|       Hodder &amp| 2000|       Hodder &amp|      Turning Thirty|        Mike Gayle|
|07434

In [ ]:
Books16.filter('publisher1 like "%&amp%"').count()

13238

In [ ]:
Books16.filter('author1 like "%&amp%"').show()

+----------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|      isbn|               title|              author|                year|           publisher|year1|          publisher1|              title1|             author1|
+----------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|0061076031|      Mary-Kate &amp| Ashley Switching...|      Mary-Kate &amp|        Ashley Olsen| null|        Ashley Olsen|Mary-Kate and  As...|      Mary-Kate &amp|
|0878911774|        Algebra &amp| Trigonometry Sup...|The Staff of Rese...| Education Associ...| null| Education Associ...|Algebra and  Trig...|The Staff of Rese...|
|1563831236|Gifts in a Jar: B...|            Brownies|              G &amp|        R Publishing| null|        R Publishing|Gifts in a Jar: B...|              G &amp|
|006

In [ ]:
Books16.filter('author1 like "%&amp%"').count()

32

In [31]:
# replacing "&amp" with 'and'
Books17 = Books16.withColumn('publisher2', f.when(
                          Books16.publisher1.like("%&amp%"),regexp_replace('publisher1', '&amp', ''))\
                           .otherwise(Books16.publisher1))


Books17.filter('publisher2 like "%&amp%"').show()

+----+-----+------+----+---------+-----+----------+------+-------+----------+
|isbn|title|author|year|publisher|year1|publisher1|title1|author1|publisher2|
+----+-----+------+----+---------+-----+----------+------+-------+----------+
+----+-----+------+----+---------+-----+----------+------+-------+----------+



In [33]:
# Creeate author 2 column in order to proceed with data cleaning
Books18 = Books17.withColumn('author2', f.when(
                          Books17.author1.like("%&amp%"),concat_ws( ' ', Books17.author1 , Books17.publisher1))\
                           .otherwise(Books17.author1))



Books18.filter('author2 like "%&amp%"').show()

+----------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|      isbn|               title|              author|                year|           publisher|year1|          publisher1|              title1|             author1|          publisher2|             author2|
+----------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|0061076031|      Mary-Kate &amp| Ashley Switching...|      Mary-Kate &amp|        Ashley Olsen| null|        Ashley Olsen|Mary-Kate and  As...|      Mary-Kate &amp|        Ashley Olsen|Mary-Kate &amp  A...|
|0878911774|        Algebra &amp| Trigonometry Sup...|The Staff of Rese...| Education Associ...| null| Education Associ...|Algebra and  Trig...|The Staff of Rese...| Ed

In [34]:
# replacing "&amp" with 'and'
Books18 = Books18.withColumn('author2', regexp_replace('author2', '&amp', 'and'))
Books18.filter('author2 like "%&amp%"').show()

+----+-----+------+----+---------+-----+----------+------+-------+----------+-------+
|isbn|title|author|year|publisher|year1|publisher1|title1|author1|publisher2|author2|
+----+-----+------+----+---------+-----+----------+------+-------+----------+-------+
+----+-----+------+----+---------+-----+----------+------+-------+----------+-------+



In [ ]:
Books18.filter('author2 == 0').count()

4441

In [35]:
# replacing zeros in author2 column with 'unknown'
Books18 = Books18.withColumn("author2", f.when(col("author2") == '0' ,'unknown').otherwise(col("author2")))

Books18.filter('author2 like "0%"').show()

+----+-----+------+----+---------+-----+----------+------+-------+----------+-------+
|isbn|title|author|year|publisher|year1|publisher1|title1|author1|publisher2|author2|
+----+-----+------+----+---------+-----+----------+------+-------+----------+-------+
+----+-----+------+----+---------+-----+----------+------+-------+----------+-------+



In [42]:
# cleaning author2 column
books19 = Books18.withColumn( 'author2', regexp_replace("author2", "[\$#,]", ''))

# Trimming 
from pyspark.sql.functions import trim, col 
books19 = books19.withColumn("author2", trim(books19.author2))\
                 .withColumn("title1", trim(books19.title1))\
                 .withColumn("publisher2", trim(books19.publisher2))

# convert column to title case 
from pyspark.sql.functions import initcap, col 
books20 = books19.select("*", initcap(col('author2')))
books20 = books20.select("*", initcap(col('title1')))
books20 = books20.select("*", initcap(col('publisher2')))

books20.show()


+----------+--------------------+--------------------+----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      isbn|               title|              author|year|           publisher|year1|          publisher1|              title1|             author1|          publisher2|             author2|    initcap(author2)|     initcap(title1)| initcap(publisher2)|
+----------+--------------------+--------------------+----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|2002|Oxford University...| 2002|Oxford University...| Classical Mythology|  Mark P. O. Morford|Oxford University...|  Mark P. O. Morford|  Mark P. O. Morford| Classical Mythology|Ox

In [43]:
# Nulls check up
books20.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in books20.columns]).show()

+----+-----+------+-----+---------+-----+----------+------+-------+----------+-------+----------------+---------------+-------------------+
|isbn|title|author| year|publisher|year1|publisher1|title1|author1|publisher2|author2|initcap(author2)|initcap(title1)|initcap(publisher2)|
+----+-----+------+-----+---------+-----+----------+------+-------+----------+-------+----------------+---------------+-------------------+
|   0|    0|     0|38846|        0|  409|         0|     0|      0|         0|      0|               0|              0|                  0|
+----+-----+------+-----+---------+-----+----------+------+-------+----------+-------+----------------+---------------+-------------------+



In [44]:
# replacing nulls in year1
books21 = books20.fillna("9999",["year1"])
books21.filter(books21.year1.isNull()).show()

+----+-----+------+----+---------+-----+----------+------+-------+----------+-------+----------------+---------------+-------------------+
|isbn|title|author|year|publisher|year1|publisher1|title1|author1|publisher2|author2|initcap(author2)|initcap(title1)|initcap(publisher2)|
+----+-----+------+----+---------+-----+----------+------+-------+----------+-------+----------------+---------------+-------------------+
+----+-----+------+----+---------+-----+----------+------+-------+----------+-------+----------------+---------------+-------------------+



In [45]:
# no nulls now
books21.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in books21.columns]).show()

+----+-----+------+-----+---------+-----+----------+------+-------+----------+-------+----------------+---------------+-------------------+
|isbn|title|author| year|publisher|year1|publisher1|title1|author1|publisher2|author2|initcap(author2)|initcap(title1)|initcap(publisher2)|
+----+-----+------+-----+---------+-----+----------+------+-------+----------+-------+----------------+---------------+-------------------+
|   0|    0|     0|38846|        0|    0|         0|     0|      0|         0|      0|               0|              0|                  0|
+----+-----+------+-----+---------+-----+----------+------+-------+----------+-------+----------------+---------------+-------------------+



In [46]:
books21.show()

+----------+--------------------+--------------------+----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      isbn|               title|              author|year|           publisher|year1|          publisher1|              title1|             author1|          publisher2|             author2|    initcap(author2)|     initcap(title1)| initcap(publisher2)|
+----------+--------------------+--------------------+----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|2002|Oxford University...| 2002|Oxford University...| Classical Mythology|  Mark P. O. Morford|Oxford University...|  Mark P. O. Morford|  Mark P. O. Morford| Classical Mythology|Ox

In [48]:
# Renaming
books22 = books21.withColumnRenamed('isbn', 'isbnBK')
books22.show()

+----------+--------------------+--------------------+----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    isbnBK|               title|              author|year|           publisher|year1|          publisher1|              title1|             author1|          publisher2|             author2|    initcap(author2)|     initcap(title1)| initcap(publisher2)|
+----------+--------------------+--------------------+----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|2002|Oxford University...| 2002|Oxford University...| Classical Mythology|  Mark P. O. Morford|Oxford University...|  Mark P. O. Morford|  Mark P. O. Morford| Classical Mythology|Ox

In [50]:
#drop columns
books23 = books22.drop('year', 'title', 'publisher', 'author','author1','author2','publisher1','publisher2', 'title1')
books23.show()

+----------+-----+--------------------+--------------------+--------------------+
|    isbnBK|year1|    initcap(author2)|     initcap(title1)| initcap(publisher2)|
+----------+-----+--------------------+--------------------+--------------------+
|0195153448| 2002|  Mark P. O. Morford| Classical Mythology|Oxford University...|
|0002005018| 2001|Richard Bruce Wright|        Clara Callan|Harperflamingo Ca...|
|0060973129| 1991|        Carlo D'este|Decision In Normandy|     Harperperennial|
|0374157065| 1999|    Gina Bari Kolata|Flu: The Story Of...|Farrar Straus Giroux|
|0393045218| 1999|     E. J. W. Barber|The Mummies Of Ur...|        W. W. Norton|
|0399135782| 1991|             Amy Tan|The Kitchen God's...|    Putnam Pub Group|
|0425176428| 2000|       Robert Cowley|What If?: The Wor...|Berkley Publishin...|
|0671870432| 1993|         Scott Turow|     Pleading Guilty|          Audioworks|
|0679425608| 1996|     David Cordingly|Under The Black F...|        Random House|
|074322678X| 200

In [51]:
# rename columns

books24 = books23.withColumnRenamed('year1', 'year')\
                  .withColumnRenamed('initcap(title1)', 'title')\
                  .withColumnRenamed('initcap(publisher2)', 'publisher')\
                  .withColumnRenamed('initcap(author2)', 'author')
books24.show()   

+----------+----+--------------------+--------------------+--------------------+
|    isbnBK|year|              author|               title|           publisher|
+----------+----+--------------------+--------------------+--------------------+
|0195153448|2002|  Mark P. O. Morford| Classical Mythology|Oxford University...|
|0002005018|2001|Richard Bruce Wright|        Clara Callan|Harperflamingo Ca...|
|0060973129|1991|        Carlo D'este|Decision In Normandy|     Harperperennial|
|0374157065|1999|    Gina Bari Kolata|Flu: The Story Of...|Farrar Straus Giroux|
|0393045218|1999|     E. J. W. Barber|The Mummies Of Ur...|        W. W. Norton|
|0399135782|1991|             Amy Tan|The Kitchen God's...|    Putnam Pub Group|
|0425176428|2000|       Robert Cowley|What If?: The Wor...|Berkley Publishin...|
|0671870432|1993|         Scott Turow|     Pleading Guilty|          Audioworks|
|0679425608|1996|     David Cordingly|Under The Black F...|        Random House|
|074322678X|2002|         An

In [52]:
# Creatimg dim_books
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window

dim_books = books24.withColumn(
    "isbnSK",
    row_number().over(Window.orderBy(monotonically_increasing_id()))
)

dim_books.show()

+----------+----+--------------------+--------------------+--------------------+------+
|    isbnBK|year|              author|               title|           publisher|isbnSK|
+----------+----+--------------------+--------------------+--------------------+------+
|0195153448|2002|  Mark P. O. Morford| Classical Mythology|Oxford University...|     1|
|0002005018|2001|Richard Bruce Wright|        Clara Callan|Harperflamingo Ca...|     2|
|0060973129|1991|        Carlo D'este|Decision In Normandy|     Harperperennial|     3|
|0374157065|1999|    Gina Bari Kolata|Flu: The Story Of...|Farrar Straus Giroux|     4|
|0393045218|1999|     E. J. W. Barber|The Mummies Of Ur...|        W. W. Norton|     5|
|0399135782|1991|             Amy Tan|The Kitchen God's...|    Putnam Pub Group|     6|
|0425176428|2000|       Robert Cowley|What If?: The Wor...|Berkley Publishin...|     7|
|0671870432|1993|         Scott Turow|     Pleading Guilty|          Audioworks|     8|
|0679425608|1996|     David Cord

In [53]:
# rearrange columns
dim_books = dim_books.select("isbnSK","isbnBK","title", "author", "year", "publisher")
dim_books.show()

+------+----------+--------------------+--------------------+----+--------------------+
|isbnSK|    isbnBK|               title|              author|year|           publisher|
+------+----------+--------------------+--------------------+----+--------------------+
|     1|0195153448| Classical Mythology|  Mark P. O. Morford|2002|Oxford University...|
|     2|0002005018|        Clara Callan|Richard Bruce Wright|2001|Harperflamingo Ca...|
|     3|0060973129|Decision In Normandy|        Carlo D'este|1991|     Harperperennial|
|     4|0374157065|Flu: The Story Of...|    Gina Bari Kolata|1999|Farrar Straus Giroux|
|     5|0393045218|The Mummies Of Ur...|     E. J. W. Barber|1999|        W. W. Norton|
|     6|0399135782|The Kitchen God's...|             Amy Tan|1991|    Putnam Pub Group|
|     7|0425176428|What If?: The Wor...|       Robert Cowley|2000|Berkley Publishin...|
|     8|0671870432|     Pleading Guilty|         Scott Turow|1993|          Audioworks|
|     9|0679425608|Under The Bla

<br>
<br>
<hr class="dashed">
<br>
<br>

# **<font color='ired'><u>Users Transformations</u> <br></font>**

In [ ]:
# Users Transformations
Users.printSchema()

root
 |-- "User-ID";"Location";"Age": string (nullable = true)



In [ ]:
Users.show()

+--------------------------+
|"User-ID";"Location";"Age"|
+--------------------------+
|                  "1";"nyc|
|             "2";"stockton|
|               "3";"moscow|
|                "4";"porto|
|          "5";"farnborough|
|         "6";"santa monica|
|           "7";"washington|
|              "8";"timmins|
|           "9";"germantown|
|            "10";"albacete|
|           "11";"melbourne|
|          "12";"fort bragg|
|           "13";"barcelona|
|          "14";"mediapolis|
|             "15";"calgary|
|         "16";"albuquerque|
|          "17";"chesapeake|
|      "18";"rio de janeiro|
|              "19";"weston|
|           "20";"langhorne|
+--------------------------+
only showing top 20 rows



In [54]:
# creating new schema for this dataset
from pyspark.sql.types import *

users_schema = [
               StructField('UserID', IntegerType(), True),
               StructField('location', StringType(), True),
               StructField('age', IntegerType(), True)
            ]

final_struc = StructType(fields = users_schema)

users = spark.read.csv(
    '/content/drive/MyDrive/Spark/BigDataProject/BX-Users.csv',
    sep=';',
    header=True,
    schema=final_struc 
)

users.printSchema()

root
 |-- UserID: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- age: integer (nullable = true)



In [ ]:
users.describe().show()

+-------+-----------------+--------------------+-----------------+
|summary|           UserID|            location|              age|
+-------+-----------------+--------------------+-----------------+
|  count|           278858|              278859|           168096|
|   mean|         139429.5|                null|34.75143370454978|
| stddev|80499.51502027822|                null|14.42809738245543|
|    min|                1|"alexandria"., "a...|                0|
|    max|           278858|��������� �,�����...|              244|
+-------+-----------------+--------------------+-----------------+



In [56]:
#splitting location column
from pyspark.sql.functions import split

split_column2 = pyspark.sql.functions.split(users['location'], ',')
users1 = users.withColumn('city', split_column2.getItem(0))\
              .withColumn('region', split_column2.getItem(1))\
              .withColumn('country', split_column2.getItem(2))

users2 = users1.drop('location')
users2.show()

+------+----+--------------+----------------+---------------+
|UserID| age|          city|          region|        country|
+------+----+--------------+----------------+---------------+
|     1|null|           nyc|        new york|            usa|
|     2|  18|      stockton|      california|            usa|
|     3|null|        moscow| yukon territory|         russia|
|     4|  17|         porto|        v.n.gaia|       portugal|
|     5|null|   farnborough|           hants| united kingdom|
|     6|  61|  santa monica|      california|            usa|
|     7|null|    washington|              dc|            usa|
|     8|null|       timmins|         ontario|         canada|
|     9|null|    germantown|       tennessee|            usa|
|    10|  26|      albacete|       wisconsin|          spain|
|    11|  14|     melbourne|        victoria|      australia|
|    12|null|    fort bragg|      california|            usa|
|    13|  26|     barcelona|       barcelona|          spain|
|    14|

In [57]:
# replacing nulls in age column with -1

users3 = users2.fillna(value = -1, subset = ['age'])
users3.show()


+------+---+--------------+----------------+---------------+
|UserID|age|          city|          region|        country|
+------+---+--------------+----------------+---------------+
|     1| -1|           nyc|        new york|            usa|
|     2| 18|      stockton|      california|            usa|
|     3| -1|        moscow| yukon territory|         russia|
|     4| 17|         porto|        v.n.gaia|       portugal|
|     5| -1|   farnborough|           hants| united kingdom|
|     6| 61|  santa monica|      california|            usa|
|     7| -1|    washington|              dc|            usa|
|     8| -1|       timmins|         ontario|         canada|
|     9| -1|    germantown|       tennessee|            usa|
|    10| 26|      albacete|       wisconsin|          spain|
|    11| 14|     melbourne|        victoria|      australia|
|    12| -1|    fort bragg|      california|            usa|
|    13| 26|     barcelona|       barcelona|          spain|
|    14| -1|    mediapol

In [58]:
#changing empties to unknown in city, region and country columns

users3.filter(users2.city == '').show()


+------+---+----+--------------+--------------------+
|UserID|age|city|        region|             country|
+------+---+----+--------------+--------------------+
|  4969| 30|    |              |                    |
|  8341| -1|    |              |            portugal|
|  9396| 44|    |              |                    |
| 10787| -1|    |              |             ireland|
| 11853| -1|    |              |           singapore|
| 12271| -1|    |        marche|               italy|
| 14859| 37|    |              |                    |
| 18483| -1|    |     groningen|         netherlands|
| 22306| -1|    |              |                    |
| 33872| -1|    |              |                    |
| 35670| 43|    |              |                    |
| 37036| -1|    |              |                 usa|
| 37139| -1|    |              |                    |
| 39005| -1|    |              | the great white ...|
| 42441| 27|    | niedersachsen|             germany|
| 46927| 47|    |           

In [59]:
# Trimming
from pyspark.sql.functions import trim, col
users4 = users3.withColumn("city", trim(users3.city))
users5 = users4.withColumn("region", trim(users4.region))
users6 = users5.withColumn("country", trim(users5.country))
users6.show()

+------+---+--------------+---------------+--------------+
|UserID|age|          city|         region|       country|
+------+---+--------------+---------------+--------------+
|     1| -1|           nyc|       new york|           usa|
|     2| 18|      stockton|     california|           usa|
|     3| -1|        moscow|yukon territory|        russia|
|     4| 17|         porto|       v.n.gaia|      portugal|
|     5| -1|   farnborough|          hants|united kingdom|
|     6| 61|  santa monica|     california|           usa|
|     7| -1|    washington|             dc|           usa|
|     8| -1|       timmins|        ontario|        canada|
|     9| -1|    germantown|      tennessee|           usa|
|    10| 26|      albacete|      wisconsin|         spain|
|    11| 14|     melbourne|       victoria|     australia|
|    12| -1|    fort bragg|     california|           usa|
|    13| 26|     barcelona|      barcelona|         spain|
|    14| -1|    mediapolis|           iowa|           us

In [60]:
# replace empties with "unknown"
from pyspark.sql.functions import col,when
users7 = users6.withColumn("city", when(col("city")=="" ,'unknown').otherwise(col("city")))
users8 = users7.withColumn("region", when(col("region")=="" ,'unknown').otherwise(col("region")))
users9 = users8.withColumn("country", when(col("country")=="" ,'unknown').otherwise(col("country")))

users9.show()

+------+---+--------------+---------------+--------------+
|UserID|age|          city|         region|       country|
+------+---+--------------+---------------+--------------+
|     1| -1|           nyc|       new york|           usa|
|     2| 18|      stockton|     california|           usa|
|     3| -1|        moscow|yukon territory|        russia|
|     4| 17|         porto|       v.n.gaia|      portugal|
|     5| -1|   farnborough|          hants|united kingdom|
|     6| 61|  santa monica|     california|           usa|
|     7| -1|    washington|             dc|           usa|
|     8| -1|       timmins|        ontario|        canada|
|     9| -1|    germantown|      tennessee|           usa|
|    10| 26|      albacete|      wisconsin|         spain|
|    11| 14|     melbourne|       victoria|     australia|
|    12| -1|    fort bragg|     california|           usa|
|    13| 26|     barcelona|      barcelona|         spain|
|    14| -1|    mediapolis|           iowa|           us

In [ ]:
users9.describe().show()

+-------+-----------------+------------------+------------------+------------------+-----------------+
|summary|           UserID|               age|              city|            region|          country|
+-------+-----------------+------------------+------------------+------------------+-----------------+
|  count|           278858|            278859|            278859|            278856|           278855|
|   mean|         139429.5|20.550937929204366|       29850.59375|20132.666666666668|          35595.0|
| stddev|80499.51502027822|20.773047999896715|34181.623164352524|31023.682490387368|36136.94102992117|
|    min|                1|                -1|     "alexandria".|                 "|            "n/a"|
|    max|           278858|               244|       ��������� �|         ���������|    ��������� ���|
+-------+-----------------+------------------+------------------+------------------+-----------------+



In [61]:
# there are 'n/a' values in region columns. not sure if we need region column for the analysis. leave it for now
users9.filter(users9.region == 'n/a').count()

12527

In [62]:
# delete unwanted characters
users10 = users9.withColumn( 'city', regexp_replace("city", "[\$#,]", ''))
users11 = users10.withColumn( 'region', regexp_replace("region", "[\$#,]", ''))
users12 = users11.withColumn( 'country', regexp_replace("country", "[\$#,]", ''))
users12.show()

+------+---+--------------+---------------+--------------+
|UserID|age|          city|         region|       country|
+------+---+--------------+---------------+--------------+
|     1| -1|           nyc|       new york|           usa|
|     2| 18|      stockton|     california|           usa|
|     3| -1|        moscow|yukon territory|        russia|
|     4| 17|         porto|       v.n.gaia|      portugal|
|     5| -1|   farnborough|          hants|united kingdom|
|     6| 61|  santa monica|     california|           usa|
|     7| -1|    washington|             dc|           usa|
|     8| -1|       timmins|        ontario|        canada|
|     9| -1|    germantown|      tennessee|           usa|
|    10| 26|      albacete|      wisconsin|         spain|
|    11| 14|     melbourne|       victoria|     australia|
|    12| -1|    fort bragg|     california|           usa|
|    13| 26|     barcelona|      barcelona|         spain|
|    14| -1|    mediapolis|           iowa|           us

In [63]:
# understanding country column
users13 = users12.groupBy(users12['country']).count()
users13.show()


+--------------------+-----+
|             country|count|
+--------------------+-----+
|            guernsey|   12|
|               aruba|    4|
|           china ���|    1|
|           kathmandu|    1|
| armed forces-europe|    1|
|     yukon territory|    1|
|usa (currently li...|    1|
|             finland|  941|
|               maroc|    1|
|               lazio|    6|
|       west virginia|    4|
|           australia|11725|
|              greece|  425|
|           ama lurra|    1|
|       new hampshire|    3|
|              61 men|    1|
|               lecce|    1|
|                cook|    1|
|            portugal| 3371|
|                 dfg|    1|
+--------------------+-----+
only showing top 20 rows



In [64]:
# converting to pandas df
import pandas as pd
import numpy as np
pdusers12 = users12.toPandas()
print(pdusers12)

          UserID  age         city           region         country
0            1.0   -1          nyc         new york             usa
1            2.0   18     stockton       california             usa
2            3.0   -1       moscow  yukon territory          russia
3            4.0   17        porto         v.n.gaia        portugal
4            5.0   -1  farnborough            hants  united kingdom
...          ...  ...          ...              ...             ...
278854  278854.0   -1     portland           oregon             usa
278855  278855.0   50       tacoma       washington  united kingdom
278856  278856.0   -1     brampton          ontario          canada
278857  278857.0   -1    knoxville        tennessee             usa
278858  278858.0   -1       dublin              n/a         ireland

[278859 rows x 5 columns]


In [ ]:
# dataprep downloading for further data transformation
!pip install dataprep

In [66]:
# Cleaning country column with clean_country function
from dataprep.clean import clean_country
pdusers13 = clean_country(pdusers12, "country",fuzzy_dist=2)
pdusers13

Country Cleaning Report:
	274074 values cleaned (98.28%)
	4690 values unable to be parsed (1.68%), set to NaN
Result contains 274074 (98.28%) values in the correct format and 4785 null values (1.72%)


,UserID,age,city,region,country,country_clean
0,1.0,-1,nyc,new york,usa,United States
1,2.0,18,stockton,california,usa,United States
2,3.0,-1,moscow,yukon territory,russia,Russia
3,4.0,17,porto,v.n.gaia,portugal,Portugal
4,5.0,-1,farnborough,hants,united kingdom,United Kingdom
...,...,...,...,...,...,...
278854,278854.0,-1,portland,oregon,usa,United States
278855,278855.0,50,tacoma,washington,united kingdom,United Kingdom
278856,278856.0,-1,brampton,ontario,canada,Canada
278857,278857.0,-1,knoxville,tennessee,usa,United States


In [67]:
# Null check up in fixed country column
pdusers13[pdusers13['country_clean'].isnull()]

,UserID,age,city,region,country,country_clean
18,19.0,14,weston,unknown,unknown,NaN
90,91.0,-1,toronto/newmarket,unknown,unknown,NaN
256,257.0,-1,quit,quit,quit,NaN
311,312.0,-1,lk,jlk,unknown,NaN
317,318.0,-1,berlin,n/a,unknown,NaN
...,...,...,...,...,...,...
278625,278625.0,36,savona,unknown,unknown,NaN
278652,278652.0,-1,murrieta,unknown,unknown,NaN
278698,278698.0,-1,pasadena,maryland,unknown,NaN
278717,278717.0,53,mayag�ez,puerto rico,unknown,NaN


In [68]:
# Comparison of  original country column when country_clean is NAN to see what kind of values didn't make it to country_clean column
x = pdusers13[(pdusers13['country_clean'].isnull()) & (pdusers13.country != 'unknown')]
x

,UserID,age,city,region,country,country_clean
256,257.0,-1,quit,quit,quit,NaN
3095,3096.0,-1,n/a,n/a,n/a,NaN
6746,6747.0,23,����,�㶫,���,NaN
9098,9099.0,-1,4232 spruce st 3f,philadelphia,19104,NaN
10646,10647.0,-1,n/a,n/a,n/a,NaN
...,...,...,...,...,...,...
271681,271682.0,-1,n/a,n/a,n/a,NaN
272826,272827.0,15,shanghai,shanghai,&20013;&22269;,NaN
274274,274275.0,-1,taiwan,taipei,yunling,NaN
275080,275081.0,-1,cernusco s,None,None,NaN


In [70]:
# back to pyspark dataset
from pyspark.sql.types import *

users_schema1 = [
               StructField('UserID', DoubleType(), True),
               StructField('age', IntegerType(), True),
               StructField('city', StringType(), True),
               StructField('region', StringType(), True), 
               StructField('country', StringType(), True),                
               StructField('country_clean', StringType(), True)
               ]
final_struc1 = StructType(fields = users_schema1)


users14 = spark.createDataFrame(pdusers13, schema = final_struc1) 
users14.show()


+------+---+--------------+---------------+--------------+--------------+
|UserID|age|          city|         region|       country| country_clean|
+------+---+--------------+---------------+--------------+--------------+
|   1.0| -1|           nyc|       new york|           usa| United States|
|   2.0| 18|      stockton|     california|           usa| United States|
|   3.0| -1|        moscow|yukon territory|        russia|        Russia|
|   4.0| 17|         porto|       v.n.gaia|      portugal|      Portugal|
|   5.0| -1|   farnborough|          hants|united kingdom|United Kingdom|
|   6.0| 61|  santa monica|     california|           usa| United States|
|   7.0| -1|    washington|             dc|           usa| United States|
|   8.0| -1|       timmins|        ontario|        canada|        Canada|
|   9.0| -1|    germantown|      tennessee|           usa| United States|
|  10.0| 26|      albacete|      wisconsin|         spain|         Spain|
|  11.0| 14|     melbourne|       vict

In [71]:
# fill nulls with 'unknown'
users15 = users14.na.fill(value='unknown',subset=["country_clean"])
users15.show()

+------+---+--------------+---------------+--------------+--------------+
|UserID|age|          city|         region|       country| country_clean|
+------+---+--------------+---------------+--------------+--------------+
|   1.0| -1|           nyc|       new york|           usa| United States|
|   2.0| 18|      stockton|     california|           usa| United States|
|   3.0| -1|        moscow|yukon territory|        russia|        Russia|
|   4.0| 17|         porto|       v.n.gaia|      portugal|      Portugal|
|   5.0| -1|   farnborough|          hants|united kingdom|United Kingdom|
|   6.0| 61|  santa monica|     california|           usa| United States|
|   7.0| -1|    washington|             dc|           usa| United States|
|   8.0| -1|       timmins|        ontario|        canada|        Canada|
|   9.0| -1|    germantown|      tennessee|           usa| United States|
|  10.0| 26|      albacete|      wisconsin|         spain|         Spain|
|  11.0| 14|     melbourne|       vict

In [72]:
# replace 'NaN' values with 'unknown' 
users16 = users15.withColumn("country_clean", when(col("country_clean")=="NaN" ,'unknown').otherwise(col("country_clean")))
users16.show()

+------+---+--------------+---------------+--------------+--------------+
|UserID|age|          city|         region|       country| country_clean|
+------+---+--------------+---------------+--------------+--------------+
|   1.0| -1|           nyc|       new york|           usa| United States|
|   2.0| 18|      stockton|     california|           usa| United States|
|   3.0| -1|        moscow|yukon territory|        russia|        Russia|
|   4.0| 17|         porto|       v.n.gaia|      portugal|      Portugal|
|   5.0| -1|   farnborough|          hants|united kingdom|United Kingdom|
|   6.0| 61|  santa monica|     california|           usa| United States|
|   7.0| -1|    washington|             dc|           usa| United States|
|   8.0| -1|       timmins|        ontario|        canada|        Canada|
|   9.0| -1|    germantown|      tennessee|           usa| United States|
|  10.0| 26|      albacete|      wisconsin|         spain|         Spain|
|  11.0| 14|     melbourne|       vict

In [73]:
#drop & rename columns
users17 = users16.drop('country')
users18 = users17.withColumnRenamed('country_clean', 'country')\
                .withColumnRenamed('UserID', 'userid')
users18.show()

+------+---+--------------+---------------+--------------+
|userid|age|          city|         region|       country|
+------+---+--------------+---------------+--------------+
|   1.0| -1|           nyc|       new york| United States|
|   2.0| 18|      stockton|     california| United States|
|   3.0| -1|        moscow|yukon territory|        Russia|
|   4.0| 17|         porto|       v.n.gaia|      Portugal|
|   5.0| -1|   farnborough|          hants|United Kingdom|
|   6.0| 61|  santa monica|     california| United States|
|   7.0| -1|    washington|             dc| United States|
|   8.0| -1|       timmins|        ontario|        Canada|
|   9.0| -1|    germantown|      tennessee| United States|
|  10.0| 26|      albacete|      wisconsin|         Spain|
|  11.0| 14|     melbourne|       victoria|     Australia|
|  12.0| -1|    fort bragg|     california| United States|
|  13.0| 26|     barcelona|      barcelona|         Spain|
|  14.0| -1|    mediapolis|           iowa| United State

In [ ]:
users18.select('country').distinct().count()

217

In [ ]:
# top user countries
grouppedbycountry = users18.groupBy('country').count()
grouppedbycountry.orderBy(grouppedbycountry['count'].desc()).show()


+--------------+------+
|       country| count|
+--------------+------+
| United States|139213|
|        Canada| 21576|
|United Kingdom| 18338|
|       Germany| 17055|
|         Spain| 13219|
|     Australia| 11738|
|         Italy| 11297|
|       unknown|  4785|
|        France|  3567|
|      Portugal|  3371|
|   New Zealand|  3099|
|   Netherlands|  3046|
|   Switzerland|  1763|
|        Brazil|  1687|
|         China|  1538|
|        Sweden|  1452|
|         India|  1345|
|       Austria|  1151|
|      Malaysia|  1105|
|     Argentina|  1090|
+--------------+------+
only showing top 20 rows



In [ ]:
users18.describe().show()

+-------+------+------------------+------------------+-----------------+-----------+
|summary|userid|               age|              city|           region|    country|
+-------+------+------------------+------------------+-----------------+-----------+
|  count|278859|            278859|            278859|           278856|     278859|
|   mean|   NaN|20.550937929204366|       29850.59375|      18888.28125|       null|
| stddev|   NaN|20.773047999896885|34181.623164352524|30403.15300842711|       null|
|    min|   1.0|                -1|     "alexandria".|                "|Afghanistan|
|    max|   NaN|               244|       ��������� �|        ���������|    unknown|
+-------+------+------------------+------------------+-----------------+-----------+



In [ ]:
# 'Nan' in UserID

users18.where(users18.userid == 'NaN').show()


+------+---+----+------+-------+
|userid|age|city|region|country|
+------+---+----+------+-------+
|   NaN| -1|NULL|  null|unknown|
+------+---+----+------+-------+



In [74]:
# delete row with userid = nan
users19 = users18.where(users18.userid !='NaN')
users19.show()

+------+---+--------------+---------------+--------------+
|userid|age|          city|         region|       country|
+------+---+--------------+---------------+--------------+
|   1.0| -1|           nyc|       new york| United States|
|   2.0| 18|      stockton|     california| United States|
|   3.0| -1|        moscow|yukon territory|        Russia|
|   4.0| 17|         porto|       v.n.gaia|      Portugal|
|   5.0| -1|   farnborough|          hants|United Kingdom|
|   6.0| 61|  santa monica|     california| United States|
|   7.0| -1|    washington|             dc| United States|
|   8.0| -1|       timmins|        ontario|        Canada|
|   9.0| -1|    germantown|      tennessee| United States|
|  10.0| 26|      albacete|      wisconsin|         Spain|
|  11.0| 14|     melbourne|       victoria|     Australia|
|  12.0| -1|    fort bragg|     california| United States|
|  13.0| 26|     barcelona|      barcelona|         Spain|
|  14.0| -1|    mediapolis|           iowa| United State

In [75]:
# Null chek up
users19.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in users19.columns]).show()

+------+---+----+------+-------+
|userid|age|city|region|country|
+------+---+----+------+-------+
|     0|  0|   0|     2|      0|
+------+---+----+------+-------+



In [ ]:
# no empties in countries
users19.filter(users19.country =='  ').show()

+------+---+----+------+-------+
|userid|age|city|region|country|
+------+---+----+------+-------+
+------+---+----+------+-------+



In [ ]:
users19.show()

+------+---+--------------+---------------+--------------+
|userid|age|          city|         region|       country|
+------+---+--------------+---------------+--------------+
|   1.0| -1|           nyc|       new york| United States|
|   2.0| 18|      stockton|     california| United States|
|   3.0| -1|        moscow|yukon territory|        Russia|
|   4.0| 17|         porto|       v.n.gaia|      Portugal|
|   5.0| -1|   farnborough|          hants|United Kingdom|
|   6.0| 61|  santa monica|     california| United States|
|   7.0| -1|    washington|             dc| United States|
|   8.0| -1|       timmins|        ontario|        Canada|
|   9.0| -1|    germantown|      tennessee| United States|
|  10.0| 26|      albacete|      wisconsin|         Spain|
|  11.0| 14|     melbourne|       victoria|     Australia|
|  12.0| -1|    fort bragg|     california| United States|
|  13.0| 26|     barcelona|      barcelona|         Spain|
|  14.0| -1|    mediapolis|           iowa| United State

In [76]:
# Renaming
users20 = users19.withColumnRenamed('userid', 'userBK')
users20.show()

+------+---+--------------+---------------+--------------+
|userBK|age|          city|         region|       country|
+------+---+--------------+---------------+--------------+
|   1.0| -1|           nyc|       new york| United States|
|   2.0| 18|      stockton|     california| United States|
|   3.0| -1|        moscow|yukon territory|        Russia|
|   4.0| 17|         porto|       v.n.gaia|      Portugal|
|   5.0| -1|   farnborough|          hants|United Kingdom|
|   6.0| 61|  santa monica|     california| United States|
|   7.0| -1|    washington|             dc| United States|
|   8.0| -1|       timmins|        ontario|        Canada|
|   9.0| -1|    germantown|      tennessee| United States|
|  10.0| 26|      albacete|      wisconsin|         Spain|
|  11.0| 14|     melbourne|       victoria|     Australia|
|  12.0| -1|    fort bragg|     california| United States|
|  13.0| 26|     barcelona|      barcelona|         Spain|
|  14.0| -1|    mediapolis|           iowa| United State

In [77]:
# Creatimg dim_books
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window

dim_users = users20.withColumn(
    "userSK",
    row_number().over(Window.orderBy(monotonically_increasing_id()))
)

dim_users.show()

+------+---+--------------+---------------+--------------+------+
|userBK|age|          city|         region|       country|userSK|
+------+---+--------------+---------------+--------------+------+
|   1.0| -1|           nyc|       new york| United States|     1|
|   2.0| 18|      stockton|     california| United States|     2|
|   3.0| -1|        moscow|yukon territory|        Russia|     3|
|   4.0| 17|         porto|       v.n.gaia|      Portugal|     4|
|   5.0| -1|   farnborough|          hants|United Kingdom|     5|
|   6.0| 61|  santa monica|     california| United States|     6|
|   7.0| -1|    washington|             dc| United States|     7|
|   8.0| -1|       timmins|        ontario|        Canada|     8|
|   9.0| -1|    germantown|      tennessee| United States|     9|
|  10.0| 26|      albacete|      wisconsin|         Spain|    10|
|  11.0| 14|     melbourne|       victoria|     Australia|    11|
|  12.0| -1|    fort bragg|     california| United States|    12|
|  13.0| 2

In [78]:
# rearrange columns and drop columns
dim_users = dim_users.select("userSK","userBK","age", "country")
dim_users.show()

+------+------+---+--------------+
|userSK|userBK|age|       country|
+------+------+---+--------------+
|     1|   1.0| -1| United States|
|     2|   2.0| 18| United States|
|     3|   3.0| -1|        Russia|
|     4|   4.0| 17|      Portugal|
|     5|   5.0| -1|United Kingdom|
|     6|   6.0| 61| United States|
|     7|   7.0| -1| United States|
|     8|   8.0| -1|        Canada|
|     9|   9.0| -1| United States|
|    10|  10.0| 26|         Spain|
|    11|  11.0| 14|     Australia|
|    12|  12.0| -1| United States|
|    13|  13.0| 26|         Spain|
|    14|  14.0| -1| United States|
|    15|  15.0| -1|        Canada|
|    16|  16.0| -1| United States|
|    17|  17.0| -1| United States|
|    18|  18.0| 25|        Brazil|
|    19|  19.0| 14|       unknown|
|    20|  20.0| 19| United States|
+------+------+---+--------------+
only showing top 20 rows



<br>
<br>
<hr class="dashed">
<br>
<br>

# **<font color='cgreen'><u>Spark SQL</u> <br></font>**


In [79]:
# creating temp views
dim_ratings.createOrReplaceTempView('ratings')
dim_books.createOrReplaceTempView('books')
dim_users.createOrReplaceTempView('users')

In [80]:
# creating fact table 
ratingres = spark.sql('select * from ratings')
ratingres.show()

+--------+------+----------+------+
|ratingSK|userid|      isbn|rating|
+--------+------+----------+------+
|       1|276725|034545104X|   0.0|
|       2|276726|0155061224|   5.0|
|       3|276727|0446520802|   0.0|
|       4|276729|052165615X|   3.0|
|       5|276729|0521795028|   6.0|
|       6|276733|2080674722|   0.0|
|       7|276736|3257224281|   8.0|
|       8|276737|0600570967|   6.0|
|       9|276744|038550120X|   7.0|
|      10|276745| 342310538|  10.0|
|      11|276746|0425115801|   0.0|
|      12|276746|0449006522|   0.0|
|      13|276746|0553561618|   0.0|
|      14|276746|055356451X|   0.0|
|      15|276746|0786013990|   0.0|
|      16|276746|0786014512|   0.0|
|      17|276747|0060517794|   9.0|
|      18|276747|0451192001|   0.0|
|      19|276747|0609801279|   0.0|
|      20|276747|0671537458|   9.0|
+--------+------+----------+------+
only showing top 20 rows



In [81]:
# creating fact table

factratings = spark.sql(
    'select ratingSK, userSK, isbnSK,  rating from ratings r join users u on r.userid = u.userBK join books b on b.isbnBK = r.isbn' 
)
factratings.show()

+--------+------+------+------+
|ratingSK|userSK|isbnSK|rating|
+--------+------+------+------+
|       1|276725|  2967|   0.0|
|       2|276726|225830|   5.0|
|       3|276727| 11055|   0.0|
|       4|276729|246855|   3.0|
|       5|276729|246856|   6.0|
|       6|276733|123646|   0.0|
|       9|276744|  9296|   7.0|
|      11|276746|  2031|   0.0|
|      12|276746|   228|   0.0|
|      13|276746|  1005|   0.0|
|      14|276746|   597|   0.0|
|      15|276746| 87285|   0.0|
|      16|276746| 30986|   0.0|
|      17|276747|  4780|   9.0|
|      18|276747| 25798|   0.0|
|      19|276747|  7155|   0.0|
|      20|276747|  1837|   9.0|
|      21|276747|  6277|   8.0|
|      22|276747|246857|   7.0|
|      23|276747| 71880|   0.0|
+--------+------+------+------+
only showing top 20 rows



In [ ]:
# 107,901 rows lost in fact ratings, because of isbn values that didn't match
factratings.count()

940674

In [82]:
dimbooks = spark.sql('select * from books')
dimbooks.show()

+------+----------+--------------------+--------------------+----+--------------------+
|isbnSK|    isbnBK|               title|              author|year|           publisher|
+------+----------+--------------------+--------------------+----+--------------------+
|     1|0195153448| Classical Mythology|  Mark P. O. Morford|2002|Oxford University...|
|     2|0002005018|        Clara Callan|Richard Bruce Wright|2001|Harperflamingo Ca...|
|     3|0060973129|Decision In Normandy|        Carlo D'este|1991|     Harperperennial|
|     4|0374157065|Flu: The Story Of...|    Gina Bari Kolata|1999|Farrar Straus Giroux|
|     5|0393045218|The Mummies Of Ur...|     E. J. W. Barber|1999|        W. W. Norton|
|     6|0399135782|The Kitchen God's...|             Amy Tan|1991|    Putnam Pub Group|
|     7|0425176428|What If?: The Wor...|       Robert Cowley|2000|Berkley Publishin...|
|     8|0671870432|     Pleading Guilty|         Scott Turow|1993|          Audioworks|
|     9|0679425608|Under The Bla

In [83]:
dimusers = spark.sql('select * from users')
dimusers.show()

+------+------+---+--------------+
|userSK|userBK|age|       country|
+------+------+---+--------------+
|     1|   1.0| -1| United States|
|     2|   2.0| 18| United States|
|     3|   3.0| -1|        Russia|
|     4|   4.0| 17|      Portugal|
|     5|   5.0| -1|United Kingdom|
|     6|   6.0| 61| United States|
|     7|   7.0| -1| United States|
|     8|   8.0| -1|        Canada|
|     9|   9.0| -1| United States|
|    10|  10.0| 26|         Spain|
|    11|  11.0| 14|     Australia|
|    12|  12.0| -1| United States|
|    13|  13.0| 26|         Spain|
|    14|  14.0| -1| United States|
|    15|  15.0| -1|        Canada|
|    16|  16.0| -1| United States|
|    17|  17.0| -1| United States|
|    18|  18.0| 25|        Brazil|
|    19|  19.0| 14|       unknown|
|    20|  20.0| 19| United States|
+------+------+---+--------------+
only showing top 20 rows



In [84]:
# Rating distribution
rating_count = dim_ratings.groupBy(["rating"]).count().orderBy(f.desc("rating"))
rating_count.show()

+------+------+
|rating| count|
+------+------+
|  10.0| 71813|
|   9.0| 62101|
|   8.0| 95037|
|   7.0| 70018|
|   6.0| 33752|
|   5.0| 46953|
|   4.0|  8031|
|   3.0|  5445|
|   2.0|  2492|
|   1.0|  1601|
|   0.0|651327|
+------+------+



In [85]:
# ratings by userid
grouppedbyuserid = dim_ratings.groupBy(['userid']).count().orderBy(f.desc('count'))
grouppedbyuserid.show()

+------+-----+
|userid|count|
+------+-----+
| 11676|13602|
|198711| 7550|
|153662| 6109|
| 98391| 5891|
| 35859| 5850|
|212898| 4785|
|278418| 4533|
| 76352| 3367|
|110973| 3100|
|235105| 3067|
|230522| 2991|
| 16795| 2948|
|234623| 2674|
| 36836| 2529|
| 52584| 2512|
|245963| 2507|
|204864| 2504|
| 55492| 2459|
|185233| 2448|
|171118| 2421|
+------+-----+
only showing top 20 rows



In [86]:
# Most rated books
grouppedbytitle = dim_books.groupBy('title').count().orderBy(f.desc('count'))
grouppedbytitle.show()

+--------------------+-----+
|               title|count|
+--------------------+-----+
|                Love|   77|
|                   I|   63|
|                  Oh|   47|
|                   1|   44|
|                 Sex|   34|
|      Selected Poems|   30|
|      Happy Birthday|   29|
|        Little Women|   26|
|             Goodbye|   25|
|                Tall|   25|
|               Hello|   24|
|          Good Night|   23|
|                  20|   22|
|   Wuthering Heights|   22|
|             Secrets|   21|
|                Baby|   21|
|               Women|   21|
|             Dracula|   21|
|                  Ut|   21|
|The Night Before ...|   20|
+--------------------+-----+
only showing top 20 rows



In [87]:
# top countries
grouppedbycountry = dim_users.groupBy('country').count().orderBy(f.desc('count'))
grouppedbycountry.show()


+--------------+------+
|       country| count|
+--------------+------+
| United States|139213|
|        Canada| 21576|
|United Kingdom| 18338|
|       Germany| 17055|
|         Spain| 13219|
|     Australia| 11738|
|         Italy| 11297|
|       unknown|  4784|
|        France|  3567|
|      Portugal|  3371|
|   New Zealand|  3099|
|   Netherlands|  3046|
|   Switzerland|  1763|
|        Brazil|  1687|
|         China|  1538|
|        Sweden|  1452|
|         India|  1345|
|       Austria|  1151|
|      Malaysia|  1105|
|     Argentina|  1090|
+--------------+------+
only showing top 20 rows



<br>
<br>
<hr class="dashed">
<br>
<br>

# **<font color='cyello'><u>Saving and downloading files</u> <br></font>**

In [88]:
#saving to files on a google drive
FolderPath = "/content/drive/MyDrive/Spark/BigDataProject"
dimusers.coalesce(1).write.option("header", "true").csv(FolderPath + '/users.csv')
dimbooks.coalesce(1).write.option("header", "true").csv(FolderPath + '/books.csv')
factratings.coalesce(1).write.option("header", "true").csv(FolderPath + '/fact.csv')


In [89]:
# Look at the output of the command using the shell command `ls`

!ls /content/drive/MyDrive/Spark/BigDataProject/users.csv
!ls /content/drive/MyDrive/Spark/BigDataProject/books.csv
!ls /content/drive/MyDrive/Spark/BigDataProject/fact.csv

part-00000-9796837a-19bc-4852-a7dc-7cb15d49cd4b-c000.csv  _SUCCESS
part-00000-ba135ed5-de09-4a49-a6d0-1c8e97b15997-c000.csv  _SUCCESS
part-00000-cae11736-75d6-472a-8d69-3425c1bb4c7f-c000.csv  _SUCCESS


In [90]:
# Rename the data files

!mv /content/drive/MyDrive/Spark/BigDataProject/users.csv/part-00000*.csv /content/drive/MyDrive/Spark/BigDataProject/users_final.csv
!mv /content/drive/MyDrive/Spark/BigDataProject/books.csv/part-00000*.csv /content/drive/MyDrive/Spark/BigDataProject/books_final.csv
!mv /content/drive/MyDrive/Spark/BigDataProject/fact.csv/part-00000*.csv /content/drive/MyDrive/Spark/BigDataProject/fact_final.csv

In [91]:
# downloading file (users_final) to a local pc

from google.colab import files
files.download('/content/drive/MyDrive/Spark/BigDataProject/users_final.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [92]:
# downloading file (books_final) to a local pc
from google.colab import files
files.download('/content/drive/MyDrive/Spark/BigDataProject/books_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [93]:
# downloading file (fact_final) to a local pc
from google.colab import files
files.download('/content/drive/MyDrive/Spark/BigDataProject/fact_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [94]:
#Stop session
spark.stop()